### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [41]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [50]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_url = f"{url}&api-key={nyt_api_key}&fq={filter_query}&bd={begin_date}&ed={end_date}&sort={sort}&fl={field_list}"


In [54]:
# Create an empty list to store the reviews
review_list = []

# loop through pages 0-19
for pages in (0,20):
    # create query with a page number
    # API results show 10 articles at a time
    query = f"&page={pages+1}"
  
    # Make a "GET" request and retrieve the JSON
    json_request = requests.get(nyt_url).json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(6)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in json_request["response"]["docs"]:
            review_list.append(doc)
        # Print the page that was just retrieved
        print(f"Page {pages}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page {pages} had no results")
        break


{'fault': {'faultstring': 'Invalid ApiKey',
  'detail': {'errorcode': 'oauth.v2.InvalidApiKey'}}}

In [49]:
# Preview the first 5 results in JSON format
first_five = review_list[0:5]

# Use json.dumps with argument indent=4 to format data
print(json.dumps(first_five,indent=4))

[]

In [45]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_list_df = pd.json_normalize(review_list)
review_list_df

""


In [44]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
review_list_df["title"]= review_list_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
review_list_df

KeyError: 'headline.main'

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
review_list_df["keywords"] = review_list_df["keywords"].apply(extract_keywords)
review_list_df

KeyError: 'keywords'

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = review_list_df["title"].to_list()
titles.head()

KeyError: 'title'

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url_tmd = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies =[]

# Create a request counter to sleep the requests after a multiple
# of 50 requests
counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if requests % 50 == 0:
        print("Sleeping in progress.")
        time.sleep(1)


    # Add 1 to the request counter
    counter = counter + 1
    
    # Perform a "GET" request for The Movie Database
    tmd_request = requests.get(url + title + tmdb_key_string)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = tmd_request.json()["results"][0]["id"]

        # Make a request for a the full movie details
        url_detail =f"{url_tmd}{movie_id}"

        # Execute "GET" request with url
        get_url = requests.get(url_detail)
        data_get_url = get_url.json()
        
        # Extract the genre names into a list
        genre_list = data_get_url["genre"]

        # Extract the spoken_languages' English name into a list
        spoken_languages = data_get_url["spoken_language"]

        # Extract the production_countries' name into a list
        country_production = data_get_url["production_conturies"]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies.append({"title": title, 
                                "original_title": data_get_url["original_title"], 
                                "budget": data_get_url["budget"],
                                "original_language": data_get_url["original_language"],
                                "homepage": data_get_url["homepage"],
                                "overview": data_get_url["overview"],
                                "popularity": data_get_url["popularity"],
                                "runtime": data_get_url["runtime"],
                                "revenue": data_get_url["revenue"],
                                "release_date": data_get_url["release_date"],
                                "vote_average": data_get_url["vote_average"],
                                "vote_count": data_get_url["vote_count"], 
                                "genres": genre_list,
                                "spoken_language": spoken_languages,
                                "production_countires": country_production})
        print("Title's found" + title)  
        # Print out the title that was found
    except:
        print(f"Title: {title} was not found.")


NameError: name 'titles' is not defined

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies[0:5], indent= 4))

[]


In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies)
tmdb_df

""


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
nyt_tmdb_df_merger = pd.merge(tmdb_df, review_list_df, how= "inner", on= "title")
nyt_tmdb_df_merger

KeyError: 'title'

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
fix_columns = ["genre" , "spoken_language", "production_countries"]

# Create a list of characters to remove
remove_characters = ["[","]", "'"]

# Loop through the list of columns to fix
for column in fix_columns: 
    # Convert the column to type 'str'
    nyt_tmdb_df_merger[column] = nyt_tmdb_df_merger[column].astype('str')

    # Loop through characters to remove
    for char in remove_characters:
        nyt_tmdb_df_merger[column] = nyt_tmdb_df_merger[column].str.replace(char, "")

# Display the fixed DataFrame
nyt_tmdb_df_merger.head()

NameError: name 'nyt_tmdb_df_merger' is not defined

In [ ]:
# Drop "byline.person" column
nyt_tmdb_df_merger.drop("byline.person", axis=1, inplace=True)
nyt_tmdb_df_merger.head()

NameError: name 'nyt_tmdb_df_merger' is not defined

In [ ]:
# Delete duplicate rows and reset index
update_nyt_tmdb_merger = nyt_tmdb_df_merger.drop_duplicates().reset_index()
update_nyt_tmdb_merger

NameError: name 'nyt_tmdb_df_merger' is not defined

In [ ]:
# Export data to CSV without the index
update_nyt_tmdb_merger.to_csv("collected_data_update.csv", index=False)

NameError: name 'update_nyt_tmdb_merger' is not defined